In [2]:
import pandas as pd
import urllib
from urllib.parse import urlparse, parse_qs

In [3]:
df = pd.read_csv('data.tsv', sep='\t')

In [4]:
sort_df = df.sort_values(['ip', 'date_time'])
sort_df

,hit_time_gmt,date_time,user_agent,ip,event_list,geo_city,geo_region,geo_country,pagename,page_url,product_list,referrer
2,1254033478,2009-09-27 06:37:58,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_4...,112.33.98.231,NaN,Salt Lake City,UT,US,Home,http://www.esshopzilla.com,NaN,http://search.yahoo.com/search?p=cd+player&tog...
1,1254033379,2009-09-27 06:36:19,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_4...,23.8.61.21,2.0,Rochester,NY,US,Zune - 32 GB,http://www.esshopzilla.com/product/?pid=asfe13,Electronics;Zune - 328GB;1;;,http://www.bing.com/search?q=Zune&go=&form=QBL...
5,1254033775,2009-09-27 06:42:55,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_4...,23.8.61.21,12.0,Rochester,NY,US,Shopping Cart,http://www.esshopzilla.com/cart/,NaN,http://www.esshopzilla.com/product/?pid=asfe13
8,1254034072,2009-09-27 06:47:52,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_4...,23.8.61.21,11.0,Rochester,NY,US,Order Checkout Details,https://www.esshopzilla.com/checkout/,NaN,http://www.esshopzilla.com/cart/
11,1254034369,2009-09-27 06:52:49,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_4...,23.8.61.21,NaN,Rochester,NY,US,Order Confirmation,https://www.esshopzilla.com/checkout/?a=confirm,NaN,https://www.esshopzilla.com/checkout/
14,1254034666,2009-09-27 06:57:46,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_4...,23.8.61.21,1.0,Rochester,NY,US,Order Complete,https://www.esshopzilla.com/checkout/?a=complete,Electronics;Zune - 32GB;1;250;,https://www.esshopzilla.com/checkout/?a=confirm
3,1254033577,2009-09-27 06:39:37,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_4...,44.12.96.2,NaN,Duncan,OK,US,Hot Buys,http://www.esshopzilla.com/hotbuys/,NaN,http://www.google.com/search?hl=en&client=fire...
6,1254033874,2009-09-27 06:44:34,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_4...,44.12.96.2,2.0,Duncan,OK,US,Ipod - Nano - 8 GB,http://www.esshopzilla.com/product/?pid=as32213,Electronics;Ipod - Nano - 8GB;1;;,http://www.esshopzilla.com/hotbuys/
9,1254034171,2009-09-27 06:49:31,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_4...,44.12.96.2,12.0,Duncan,OK,US,Shopping Cart,http://www.esshopzilla.com/cart/,NaN,http://www.esshopzilla.com/product/?pid=as23233
12,1254034468,2009-09-27 06:54:28,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_4...,44.12.96.2,11.0,Duncan,OK,US,Order Checkout Details,https://www.esshopzilla.com/checkout/,NaN,http://www.esshopzilla.com/cart/


In [23]:
transformed_df = sort_df.drop(['hit_time_gmt', 'user_agent', 'geo_region', 'geo_country', 'geo_city', 'pagename'], axis=1)
#transformed_df

In [24]:
transformed_df.set_index('ip', inplace=True)
#transformed_df

In [12]:
search_engine_list = []
search_keyword_list = []
revenue_list = []

for row in transformed_df.itertuples():
    referrer_url = row.referrer
    parsed_url = urlparse(referrer_url)
    search_engine = parsed_url.hostname
    search_engine_list.append(search_engine)
    qr = parse_qs(parsed_url.query)
    
    if qr:
        try:
            search_keyword = qr['q'][0]
        except KeyError:
            search_keyword = ""
    else:
        search_keyword = ""
    search_keyword_list.append(search_keyword)   
    
    event_list = row.event_list
    if event_list == 1.0:
        revenue = row.product_list.split(';')[3]
        revenue_list.append(revenue)
    else:
        revenue = 0
        revenue_list.append(revenue)
                
transformed_df['search_engine'] = search_engine_list
transformed_df['search_keyword'] = search_keyword_list
transformed_df['revenue'] = revenue_list
#transformed_df

In [13]:
revenue_df = transformed_df.drop(['date_time', 'event_list', 'page_url', 'product_list', 'referrer', 
                                         'search_engine', 'search_keyword'], axis=1)
revenue_df = revenue_df[revenue_df.revenue != 0 ]
revenue_df

,revenue
ip,
23.8.61.21,250
44.12.96.2,190
67.98.123.1,290


In [14]:
search_df = transformed_df.drop(['date_time', 'event_list', 'page_url', 'product_list', 'referrer', 
                                         'revenue'], axis=1)
search_df = search_df[(search_df.search_engine != "") & (search_df.search_keyword != "")]
search_df

,search_engine,search_keyword
ip,,
23.8.61.21,www.bing.com,Zune
44.12.96.2,www.google.com,ipod
67.98.123.1,www.google.com,Ipod


In [15]:
merged_df = search_df.merge(revenue_df, left_index=True, right_index=True, how='inner')
merged_df

,search_engine,search_keyword,revenue
ip,,,
23.8.61.21,www.bing.com,Zune,250
44.12.96.2,www.google.com,ipod,190
67.98.123.1,www.google.com,Ipod,290


In [16]:
output_df = merged_df.reset_index(drop=True)
output_df = output_df.sort_values(['revenue'], ascending=False)
output_df

,search_engine,search_keyword,revenue
2,www.google.com,Ipod,290
0,www.bing.com,Zune,250
1,www.google.com,ipod,190


In [17]:
output_df = output_df.rename({'search_engine': 'Search Engine Domain', 
                              'search_keyword': 'Search Keyword',
                              'revenue': 'Revenue'}, axis=1)
output_df

,Search Engine Domain,Search Keyboard,Revenue
2,www.google.com,Ipod,290
0,www.bing.com,Zune,250
1,www.google.com,ipod,190


In [18]:
from datetime import date
today = date.today()
dte = today.strftime("%Y-%m-%d")
print(dte)

2021-12-15


In [19]:
output_df.to_csv(dte + '_SearchKeywordPerformance.tab', sep='\t', index=False)